# meerTOD Examples

In [ ]:
import numpy as np
from meerTOD import meerTODsim, example_scan, example_beam_map
import matplotlib.pyplot as plt
import healpy as hp

## Quick Start

In [ ]:
# Initialize the simulator with MeerKAT coordinates
simulator = meerTODsim(
    ant_latitude_deg=-30.7130,   # MeerKAT latitude
    ant_longitude_deg=21.4430,   # MeerKAT longitude
    ant_height_m=1054,           # MeerKAT altitude
    nside=64                    # HEALPix resolution
)

# Generate a simple scanning pattern
time_list, azimuth_list = example_scan()

# Simulate TOD for multiple frequencies
freq_list = [950, 1000, 1050]  # MHz
tod_array, sky_tod, gain_noise = simulator.generate_TOD(
    freq_list=freq_list,
    time_list=time_list[:100],  # Simulate only the first 100 times
    azimuth_deg_list=azimuth_list,
    elevation_deg=41.5
)

print(f"Generated TOD shape: {tod_array.shape}")  # (3, n_time)

## Example 1: Basic TOD Simulation

In [ ]:
# Initialize simulator
sim = meerTODsim(nside=64)  # Lower resolution for speed

# Generate scanning pattern
time_list, az_list = example_scan(dt=1.0)

# Single frequency simulation
tod, sky_tod, gain_noise = sim.generate_TOD(
    freq_list=[1000],  # 1 GHz
    time_list=time_list[:100],  # First 100 time points
    azimuth_deg_list=az_list[:100],
    elevation_deg=45.0
)

## Example 2: Multi-frequency Simulation

In [ ]:
# Wide frequency range
frequencies = np.linspace(900, 1000, 11)  # 11 channels

tod_multifreq, sky_multifreq, _ = sim.generate_TOD(
    freq_list=frequencies,
    time_list=time_list[:50],
    azimuth_deg_list=az_list[:50],
    elevation_deg=60.0,
    gain_noise_params=[1e-5, 1e-3, 1.8]  # Custom noise parameters
)

# Plot frequency-time waterfall
plt.figure(figsize=(10, 6))
plt.imshow(sky_multifreq, aspect='auto', origin='lower',
           extent=[0, len(time_list[:50]), frequencies[0], frequencies[-1]])
plt.colorbar(label='Temperature [K]')
plt.xlabel('Time sample')
plt.ylabel('Frequency [MHz]')
plt.title('Sky TOD - Frequency vs Time')
plt.show()

## Example 3: Custom Beam and Sky Models

In [ ]:
def custom_beam(freq, nside):
    """Custom frequency-dependent beam"""
    # Beam size scales with frequency
    fwhm = 70 / freq  # degrees, typical radio telescope scaling
    return example_beam_map(freq, nside, FWHM_major=fwhm, FWHM_minor=fwhm*0.8)

def point_source_sky(freq, nside):
    """Sky with a single point source"""
    npix = hp.nside2npix(nside)
    sky = np.zeros(npix)
    
    # Add point source at specific coordinates
    ra, dec = 180.0, -30.0  # degrees
    theta = np.pi/2 - np.radians(dec)
    phi = np.radians(ra)
    
    ipix = hp.ang2pix(nside, theta, phi)
    sky[ipix] = 100.0  # 100 K source
    
    return sky

# Use custom models
sim_custom = meerTODsim(
    beam_func=custom_beam,
    sky_func=point_source_sky,
    nside=64
)

# Simulate with custom models
tod_custom, _, _ = sim_custom.generate_TOD(
    freq_list=[1000],
    time_list=time_list[:100],
    azimuth_deg_list=az_list,
    elevation_deg=50.0
)